In [1]:
import pandas as pd
import numpy as np 
from keras.layers import LSTM , Dense
from keras.models import Sequential
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error
import math

Using TensorFlow backend.


In [2]:
df = pd.read_csv('FinalDataRL.csv')

In [3]:
df['ExpirationTime'] = pd.to_datetime('2018-04-26 15:29:00')

In [4]:
df['ExpirationTime'] = df['ExpirationTime'] - pd.to_datetime(df['DateTime'])

In [5]:
import datetime
df['ExpirationTime'] = df['ExpirationTime'].apply(lambda x : x.total_seconds()/60.0)

In [6]:
#Buy: 0 ,Not hold : 1 , Hold: 2 , Sell: 3 
for index,row in df.iterrows():
    if row['Action'] == -1:
        #print(row['Action'])
        df.set_value(index , 'Action' , 1)
    elif row['Action'] == 0:
        #print(row['Action'])
        df.set_value(index , 'Action' , 2)
    elif row['Action'] == 1:
        #print(row['Action'])
        df.set_value(index , 'Action' , 3)
    elif row['Action'] == 2:
        #print(row['Action'])
        df.set_value(index , 'Action' , 0)

In [7]:
#df[df['Action'] == -1] = 0 #0===> 1 (Hold) , -1 ===> 0 (#not holding), 1 ====> 2 (Sell) , 2 ===> 3 (Buy) 

In [8]:
strikePrice = 10000
df['Open_Nifty'] = df['Open_Nifty']/strikePrice
df['High_Nifty'] = df['High_Nifty']/strikePrice
df['Low_Nifty'] = df['Low_Nifty']/strikePrice
df['Close_Nifty'] = df['Close_Nifty']/strikePrice

In [9]:
data1 = df[['Open_Nifty', 'High_Nifty', 'Low_Nifty' , 'Close_Nifty' ,'TradingPosition' , 'ExpirationTime']]
data2 = df[['Close_Option' , 'Action']]

In [10]:
from sklearn.preprocessing import RobustScaler as RS
data3 = data1[['TradingPosition']]
scaler = RS()
data1 = scaler.fit_transform(data1[['Open_Nifty', 'High_Nifty', 'Low_Nifty' , 'Close_Nifty' ,'ExpirationTime']])

scaler1 = RS()
data4 = scaler1.fit_transform(data2['Close_Option'].values.reshape(-1, 1))

In [11]:
data1 = pd.DataFrame(data1)

In [12]:
data1.columns = ['Open_Nifty', 'High_Nifty', 'Low_Nifty' , 'Close_Nifty' ,'ExpirationTime']

In [13]:
data1['TradingPosition'] = data3['TradingPosition']

In [14]:
data4 = pd.DataFrame(data4)

In [15]:
data4.columns = ['Close_Option']
data4['Action'] = data2['Action']

In [16]:
timesteps1 = 50
timesteps2 = 5
dataX = []
dataY = []
for i in range(len(df.values) - timesteps1 + 1 ):
    x = data1.values[i:(i+timesteps1) , :]
    dataX.append(x)
for i in range(len(df.values) - timesteps2 + 1):
    y = data4.values[i:(i+timesteps2) , :]
    dataY.append(y)
X = np.array(dataX)
Y = np.array(dataY)

In [17]:
X = np.reshape(X , (X.shape[0] , X.shape[1]*X.shape[2]))
Y = np.reshape(Y , (Y.shape[0] , Y.shape[1]*Y.shape[2]))

In [18]:
X = X[:-timesteps2]

In [19]:
Y = Y[timesteps1:]

In [20]:
del(dataX)
del(dataY)
del(data1)
del(data2)
del(data3)
del(data4)
del(df)

https://machinelearningmastery.com/time-series-prediction-lstm-recurrent-neural-networks-python-keras/


In [21]:
Y = pd.DataFrame(Y)
X = pd.DataFrame(X)

In [22]:
X.head(4)

,0,1,2,3,4,5,6,7,8,9,...,290,291,292,293,294,295,296,297,298,299
0,0.412047,0.45888,0.415227,0.458707,1.281435,1.0,0.459789,0.46560,0.452975,0.467670,...,0.619002,0.616517,1.272848,1.0,0.616469,0.61952,0.617402,0.622279,1.272808,1.0
1,0.459789,0.46560,0.452975,0.467670,1.281395,0.0,0.471323,0.50560,0.474408,0.506722,...,0.617402,0.622279,1.272808,1.0,0.622557,0.62144,0.619002,0.618118,1.272768,1.0
2,0.471323,0.50560,0.474408,0.506722,1.281355,0.0,0.508170,0.52096,0.511196,0.515685,...,0.619002,0.618118,1.272768,1.0,0.614547,0.61888,0.614203,0.619718,1.272606,1.0
3,0.508170,0.52096,0.511196,0.515685,1.281314,1.0,0.505607,0.51744,0.507038,0.515365,...,0.614203,0.619718,1.272606,1.0,0.624479,0.64384,0.618362,0.634763,1.271921,1.0


In [23]:
Y.head(5)

,0,1,2,3,4,5,6,7,8,9
0,1.011887,2.0,1.024532,2.0,1.030096,2.0,1.009358,2.0,1.006323,2.0
1,1.024532,2.0,1.030096,2.0,1.009358,2.0,1.006323,2.0,1.009358,2.0
2,1.030096,2.0,1.009358,2.0,1.006323,2.0,1.009358,2.0,1.006323,2.0
3,1.009358,2.0,1.006323,2.0,1.009358,2.0,1.006323,2.0,1.010116,2.0
4,1.006323,2.0,1.009358,2.0,1.006323,2.0,1.010116,2.0,1.049823,2.0


In [24]:
#Train Test Split - 20% data 
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.15, random_state=42)

In [25]:
del(X)
del(Y)

In [26]:
X_train.shape 

(6377, 300)

Reshaping the input to the LSTM 

In [27]:
X_train1 = np.reshape(X_train.values , (X_train.shape[0] , timesteps1 , 6))

In [28]:
del(X_train)

In [29]:
X_test1 = np.reshape(X_test.values , (X_test.shape[0] , timesteps1 , 6))
del(X_test)

https://stackoverflow.com/questions/38714959/understanding-keras-lstms
Basic Idea to implement the LSTM-RL - Have to modify 

In [30]:
#LSTM Network 
from keras.layers import Input , RepeatVector ,TimeDistributed , Bidirectional , Activation , Flatten , concatenate , Dropout
from keras.models import Sequential, Model

m_next = timesteps2
prev = timesteps1 


In [31]:
from sklearn.model_selection import GridSearchCV
epoch = 10
batch_size = 15
latent_dim_1 = 40
latent_dim_2 = 40


In [32]:
from keras.losses import mean_squared_error,categorical_crossentropy
from keras import backend as K
from keras.metrics import categorical_accuracy

def weighted_categorical_crossentropy(weights , y_true, y_pred):
    weights = K.variable(value=weights)
    # calc
    loss = y_true * K.log(y_pred) * weights
    loss = -K.sum(loss, axis=-1)
    return loss
    

In [33]:
def wLoss(yTrue,yPred):
    weights = np.array([30.0, 1.0 ,1.0,30.0])
    #Value Function
    vfTrue = yTrue[:,:,0]
    vfPred = yPred[:,:,0]
    acTrue = yTrue[:,:,1:]
    acPred = yPred[:,:,1:]
    indexTrue = K.cast(K.argmax(acTrue , axis=-1) , 'float32')
    indexPred = K.cast(K.argmax(acPred , axis=-1) , 'float32')
    
    indexTrue=np.where(indexTrue==0,-5.00,indexTrue)
    indexTrue=np.where(indexTrue==1,-4.00,indexTrue)
    indexTrue=np.where(indexTrue==2, -3.00 ,indexTrue)
    indexTrue=np.where(indexTrue==3,-2.00,indexTrue)
    indexTrue=indexTrue+4
    
    indexPred=np.where(indexPred==0,-5.00,indexPred)
    indexPred=np.where(indexPred==1,-4.00,indexPred)
    indexPred=np.where(indexPred==2,-3.00,indexPred)
    indexPred=np.where(indexPred==3,-2.00,indexPred)
    indexPred=indexPred+4
    
    loss1=K.mean(K.square(indexTrue*vfTrue - indexPred*vfPred), axis=-1)
    #loss1 = (K.dot(indexTrue , K.transpose(vfTrue)) - K.dot(indexPred , K.transpose(vfPred)))**2
    loss2 = K.mean(weighted_categorical_crossentropy(weights , acTrue ,acPred) , axis=-1)
    loss3 = K.mean(K.square(vfTrue - vfPred), axis=-1)
    
    #return loss1+loss2+loss3 
    return (loss1+loss2 + loss3)/3

In [34]:
a = K.variable([[0, 1,2,3 ,0] ,[0,0,0,1,2] , [0,1,2 , 1,2]])
zeros = K.zeros_like(a)
b = K.variable([[0,1,2,1 ,1] ,[0,2,2,1,2] , [0,2,2 , 2,2]])
print(K.eval(zeros))
print(K.eval(K.sum(K.switch(K.equal(a , b) , a , zeros))))
print(10/12)

[[ 0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.]]
10.0
0.8333333333333334


In [35]:
K.eval(K.equal(a,b))

array([[ True,  True,  True, False, False],
       [ True, False, False,  True,  True],
       [ True, False,  True, False,  True]], dtype=bool)

In [36]:
K.eval(K.mean(K.equal(a,b)))

0.60000002

In [37]:
def valueFuncAccuracy(yTrue,yPred):
    vfTrue = yTrue[:,:,0]
    vfPred = yPred[:,:,0]
    SS_res =  K.sum(K.square(vfTrue - vfPred))
    SS_tot = K.sum(K.square( vfTrue - K.mean(vfTrue) ) )
    return ( 1 - SS_res/(SS_tot + K.epsilon()) )

def actionsAccuracy(yTrue , yPred):
    acTrue = yTrue[:,:,1:]
    acPred = yPred[:,:,1:]
    return categorical_accuracy(acTrue, acPred)

def actionsWeightedAccuracy(yTrue , yPred):
    weights = np.array([30.0, 1.0 ,1.0,30.0])
    weights = weights / np.sqrt(np.sum(weights**2))
    acTrue = yTrue[:,:,1:]
    acPred = yPred[:,:,1:]
    indexTrue = K.cast(K.argmax(acTrue , axis=-1) , 'float32') 
    indexPred = K.cast(K.argmax(acPred , axis=-1) , 'float32') 
    
    zeroTrue = K.zeros_like(indexTrue)
    onesTrue = K.ones_like(indexTrue)
    indexTrue=np.where(indexTrue==0,weights[0],indexTrue)
    indexTrue=np.where(indexTrue==1,weights[1],indexTrue)
    indexTrue=np.where(indexTrue==2,weights[2],indexTrue)
    indexTrue=np.where(indexTrue==3,weights[3],indexTrue)
    indexTrue=indexTrue+0
    
    indexPred=np.where(indexPred==0,weights[0],indexPred)
    indexPred=np.where(indexPred==1,weights[1],indexPred)
    indexPred=np.where(indexPred==2,weights[2],indexPred)
    indexPred=np.where(indexPred==3,weights[3],indexPred)
    indexPred=indexPred+0
    
    
    return K.sum(K.switch(K.equal(indexTrue , indexPred) , indexTrue , zeroTrue))/K.sum(indexTrue) 

In [38]:
input_layer = Input(shape=(prev , X_train1.shape[2]))
encoder = LSTM(latent_dim_1)(input_layer)
encoder = Dropout(0.2)(encoder)
encoder = RepeatVector(timesteps2)(encoder)
decoder = LSTM(latent_dim_2,return_sequences=True)(encoder)
decoder = Dropout(0.2)(decoder)
valueFunc = TimeDistributed(Dense(1, activation='linear'))(decoder)
actions = TimeDistributed(Dense(4, activation='softmax'))(decoder)
merge = concatenate([valueFunc, actions])
model = Model(inputs=input_layer, outputs=merge)
model.compile(loss=wLoss , optimizer='adam', metrics=[valueFuncAccuracy , actionsAccuracy , actionsWeightedAccuracy])

In [39]:
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 50, 6)        0                                            
__________________________________________________________________________________________________
lstm_1 (LSTM)                   (None, 40)           7520        input_1[0][0]                    
__________________________________________________________________________________________________
dropout_1 (Dropout)             (None, 40)           0           lstm_1[0][0]                     
__________________________________________________________________________________________________
repeat_vector_1 (RepeatVector)  (None, 5, 40)        0           dropout_1[0][0]                  
__________________________________________________________________________________________________
lstm_2 (LS

In [40]:
y_train1 = y_train.loc[: , np.arange(0 , 2*timesteps2-1,2 )] #Value Function

In [41]:
y_train2 = y_train.loc[: , np.arange(1,2*timesteps2,2)] #Actions 

In [42]:
from keras.utils import to_categorical
one_hot_labels = to_categorical(y_train2, num_classes=4)
#Buy: 0 ,Not hold : 1 , Hold: 2 , Sell: 3 
#[0,0,1,0] => Hold , [0,1,0,0] => Not Hold , 
#[0,0,0,1] => Sell , [1,0,0,0] => Buy

In [43]:
one_hot_labels.shape

(6377, 5, 4)

In [44]:
y_train1 = np.reshape(y_train1.values , (y_train1.shape[0] , y_train1.shape[1] , 1))

In [45]:
y_train1.shape

(6377, 5, 1)

In [46]:
y_train3 = np.concatenate((y_train1, one_hot_labels), axis=2)

In [47]:
from numpy.random import seed
seed(1)

In [48]:
model.fit(X_train1, y_train3, epochs=epoch , batch_size=batch_size , validation_split = 0.05)

Train on 6058 samples, validate on 319 samples
Epoch 1/10
6058/6058 [==============================] - 20s 3ms/step - loss: 1.2267 - valueFuncAccuracy: 0.9012 - actionsAccuracy: 0.5189 - actionsWeightedAccuracy: 0.5583 - val_loss: 0.9424 - val_valueFuncAccuracy: 0.9834 - val_actionsAccuracy: 0.7699 - val_actionsWeightedAccuracy: 0.8081
Epoch 2/10
6058/6058 [==============================] - 19s 3ms/step - loss: 0.8281 - valueFuncAccuracy: 0.9501 - actionsAccuracy: 0.6798 - actionsWeightedAccuracy: 0.6941 - val_loss: 0.8646 - val_valueFuncAccuracy: 0.9841 - val_actionsAccuracy: 0.7994 - val_actionsWeightedAccuracy: 0.8293
Epoch 3/10
6058/6058 [==============================] - 15s 3ms/step - loss: 0.7517 - valueFuncAccuracy: 0.9542 - actionsAccuracy: 0.6803 - actionsWeightedAccuracy: 0.6938 - val_loss: 0.8443 - val_valueFuncAccuracy: 0.9828 - val_actionsAccuracy: 0.7398 - val_actionsWeightedAccuracy: 0.7885
Epoch 4/10
6058/6058 [==============================] - 15s 3ms/step - loss: 0.7

In [49]:
#test 
y_test.shape
y_test1 = y_test.loc[: , np.arange(0 , 2*timesteps2-1,2 )] #Value Function
y_test2 = y_test.loc[: , np.arange(1, 2*timesteps2,2)] #Actions 
one_hot_labels1 = to_categorical(y_test2, num_classes=4)
y_test1 = np.reshape(y_test1.values , (y_test1.shape[0] , y_test1.shape[1] , 1))
y_test3 = np.concatenate((y_test1, one_hot_labels1), axis=2)
pred4 = model.predict(X_test1)

In [50]:
predValue = pred4[: , : , 0]
predAction = pred4[: ,:, 1:]

In [51]:
y_test3.shape

(1126, 5, 5)

In [52]:
#RMSE 
K.eval(K.mean(K.square(y_test3[:,:,0] - predValue), axis=-1))

array([ 0.00077847,  0.00162963,  0.00023069, ...,  0.00394268,
        0.00014265,  0.00556128])

In [53]:
#Categorical Accuracy 
K.eval(K.mean(K.equal(K.argmax(y_test3[:,:,1:], axis=-1), K.argmax(predAction, axis=-1))))

0.80177617

In [54]:
#weighted categorical accoracy 
weights = np.array([30.0, 1.0 ,1.0,30.0])
weights = weights / np.sqrt(np.sum(weights**2))
acTrue = y_test3[:,:,1:]
acPred = predAction
indexTrue = K.cast(K.argmax(acTrue , axis=-1) , 'float32') 
indexPred = K.cast(K.argmax(acPred , axis=-1) , 'float32') 

zeroTrue = K.zeros_like(indexTrue)
onesTrue = K.ones_like(indexTrue)
indexTrue=np.where(indexTrue==0,weights[0],indexTrue)
indexTrue=np.where(indexTrue==1,weights[1],indexTrue)
indexTrue=np.where(indexTrue==2,weights[2],indexTrue)
indexTrue=np.where(indexTrue==3,weights[3],indexTrue)
indexTrue=indexTrue+0

indexPred=np.where(indexPred==0,weights[0],indexPred)
indexPred=np.where(indexPred==1,weights[1],indexPred)
indexPred=np.where(indexPred==2,weights[2],indexPred)
indexPred=np.where(indexPred==3,weights[3],indexPred)
indexPred=indexPred+0

print(K.eval(K.sum(K.switch(K.equal(indexTrue , indexPred) , indexTrue , zeroTrue))/K.sum(indexTrue)))

0.837629


In [55]:
from sklearn.metrics import r2_score

r2_score(y_test3[:,:,0] , pred4[: , : , 0])

0.97496069777433247